In [41]:
#https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html?highlight=autoreload
%load_ext autoreload
%autoreload 2

import json
import time
from datetime import datetime
import pandas as pd
from Kafka import KafkaWriter
from OpenWeatherMap import OpenWeatherMap

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
openWeatherMap = OpenWeatherMap()
kafkaForecast = KafkaWriter('weather.forecast')
kafkaCleaned = KafkaWriter('weather.cleaned')

def format_date(dt: str) -> str:
    return datetime.fromtimestamp(float(dt)).strftime('%d.%m.%Y %H:%M')


# load 'locations.json' into a json-object and return it
def load_locations() -> json:
    with open('locations.json', mode='r') as file:
        return json.load(file)

def transform_forecast(forecast: json) -> json:
    value = {
        'city': forecast['city']['name'],
        'weather': []
    }

    preds = []
    for pred in forecast['list']:
        preds.append({
            'dt': pred['dt_txt'],
            'temp': pred['main']['temp']
        })
    value['weather'] = preds
    return value

# for each location, query openWeatherMap for the 5-day forecast and 
# store the the returned values in Kafka
def collect_forecast_data() -> None:
    cities = load_locations()
    forecasts = {}
    for key in cities:
        forecasts[key] = None

    while True:
        print("Starting collection ...")
        for key in cities:
            city = cities[key]
            forecast = transform_forecast(openWeatherMap.get_forecast(city))

            if(forecasts[key] != forecast):
                kafkaCleaned.store(key, forecast)
                forecasts[key]=forecast
                print("New forecast found!")

            kafkaForecast.store(key, forecast)
        print(f"Weather data collected at {format_date(datetime.timestamp(datetime.now()))}!")
        time.sleep(60)

In [47]:
try:
    collect_forecast_data()
except KeyboardInterrupt:
    print("... collection stopped!")

Starting collection ...
New forecast found!
New forecast found!
Weather data collected at 22.08.2022 22:59!
Starting collection ...
Weather data collected at 22.08.2022 23:00!
... collection stopped!
